# Tarea 1

In [1]:
import nltk
nltk.download('wordnet')
nltk.download('omw')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
print(wn.langs())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
['eng', 'als', 'arb', 'bul', 'cat', 'cmn', 'dan', 'ell', 'eus', 'fas', 'fin', 'fra', 'glg', 'heb', 'hrv', 'ind', 'ita', 'jpn', 'nld', 'nno', 'nob', 'pol', 'por', 'qcn', 'slv', 'spa', 'swe', 'tha', 'zsm']


In [2]:
def desambiguar(frase, palabra):
  sentidos = wn.synsets(palabra)
  mejorSentido = sentidos[0] # Sentido más frecuente
  maxSuperposicion = 0
  stop_words = set(stopwords.words('english'))
  word_tokens = set(word_tokenize(frase))
  contexto = [] # conjunto de palabras en la frase, sin palabras vacías
  for w in word_tokens:
    if w not in stop_words:
        contexto.append(w)
  contexto = set(contexto)

  for sentido in sentidos:
    # Sacar la signatura
    aux = word_tokenize(sentido.definition()) # definición del sentido
    for ejemplo in sentido.examples(): # ejemplos del sentido 
      aux += word_tokenize(ejemplo)
    signatura = []
    for x in aux:
      if x not in stop_words:
        signatura.append(x)
    signatura = set(signatura)    
    # Cálculo superposición    
    superposicion = len(signatura.intersection(contexto))
    if superposicion > maxSuperposicion:
      maxSuperposicion = superposicion
      mejorSentido = sentido
  
  print(mejorSentido.definition())
  return mejorSentido

In [3]:
desambiguar('Yesterday I went to the bank to withdraw the money and the credit card did not work', 'bank')

a financial institution that accepts deposits and channels the money into lending activities


Synset('depository_financial_institution.n.01')

In [4]:
desambiguar("The river overflowed the bank.",'bank')

sloping land (especially the slope beside a body of water)


Synset('bank.n.01')

In [5]:
desambiguar("The van pulled up outside the bank and three masked men got out.",'bank')

sloping land (especially the slope beside a body of water)


Synset('bank.n.01')

In [6]:
desambiguar("The boy leapt from the bank into the cold water.",'bank')

sloping land (especially the slope beside a body of water)


Synset('bank.n.01')

In [7]:
desambiguar("I went fishing for some sea bass.",'bass')

having or denoting a low vocal or instrumental range


Synset('bass.s.01')

In [8]:
desambiguar("The bass line of the song is too weak.",'bass')

having or denoting a low vocal or instrumental range


Synset('bass.s.01')

In [9]:
desambiguar("I can hear bass frequency sound.",'bass')

having or denoting a low vocal or instrumental range


Synset('bass.s.01')

In [10]:
desambiguar("He likes to eat grilled bass fish.",'bass')

the lean flesh of a saltwater fish of the family Serranidae


Synset('sea_bass.n.01')

# Tarea 2

In [13]:
from scipy.spatial import distance
import gensim
import numpy as np
from nltk.data import find
nltk.download('word2vec_sample')

word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)
vector_mean = model.vectors.mean(axis=0)

[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Unzipping models/word2vec_sample.zip.


In [61]:
def desambiguar_embeddings(frase, palabra):
  sentidos = wn.synsets(palabra)
  mejorSentido = sentidos[0] # Sentido más frecuente
  minDistancia = 0 # En este caso la minimizaremos, ya que queremos reducir la distancia
  stop_words = set(stopwords.words('english'))
  word_tokens = set(word_tokenize(frase))
  contexto = [] # conjunto de palabras en la frase, sin palabras vacías
  for w in word_tokens:
    if w not in stop_words:
        contexto.append(w)
  contexto = set(contexto)

  contextoMatriz = np.zeros((len(contexto),300))
  i= 0
  for palabra in contexto:
    if palabra in model.vocab:
      contextoMatriz[i]=model[palabra]
    else:
      contextoMatriz[i]=vector_mean
    i+=1
  vectorContexto = contextoMatriz.mean(axis=0)

  for sentido in sentidos:
    # Sacar la signatura
    aux = word_tokenize(sentido.definition()) # definición del sentido
    for ejemplo in sentido.examples(): # ejemplos del sentido 
      aux += word_tokenize(ejemplo)
    signatura = []
    for x in aux:
      if x not in stop_words:
        signatura.append(x)
    signatura = set(signatura)
    
    matrizSignatura = np.zeros((len(signatura),300))
    i= 0
    for palabra in signatura:
      if palabra in model.vocab:
        matrizSignatura[i]=model[palabra]
      else:
        matrizSignatura[i]=vector_mean
      i+=1
    vectorSignatura = matrizSignatura.mean(axis=0)    

    # Cálculo distancia    
    distancia = distance.cosine(vectorContexto, vectorSignatura)
    if distancia < minDistancia or minDistancia == 0:  # Cuando vale 0, la inicializamos a la primera distancia
      minDistancia = distancia
      mejorSentido = sentido

  print(mejorSentido.definition())
  return mejorSentido

In [62]:
desambiguar_embeddings('Yesterday I went to the bank to withdraw the money and the credit card did not work', 'bank')

put into a bank account


Synset('deposit.v.02')

In [63]:
desambiguar_embeddings("The river overflowed the bank.",'bank')

sloping land (especially the slope beside a body of water)


Synset('bank.n.01')

In [64]:
desambiguar_embeddings("The van pulled up outside the bank and three masked men got out.",'bank')

a container (usually with a slot in the top) for keeping money at home


Synset('savings_bank.n.02')

In [65]:
desambiguar_embeddings("The boy leapt from the bank into the cold water.",'bank')

sloping land (especially the slope beside a body of water)


Synset('bank.n.01')

In [66]:
desambiguar_embeddings("I went fishing for some sea bass.",'bass')

the lean flesh of a saltwater fish of the family Serranidae


Synset('sea_bass.n.01')

In [67]:
desambiguar_embeddings("The bass line of the song is too weak.",'bass')

having or denoting a low vocal or instrumental range


Synset('bass.s.01')

In [68]:
desambiguar_embeddings("I can hear bass frequency sound.",'bass')

having or denoting a low vocal or instrumental range


Synset('bass.s.01')

In [69]:
desambiguar_embeddings("He likes to eat grilled bass fish.",'bass')

the lean flesh of a saltwater fish of the family Serranidae


Synset('sea_bass.n.01')